## Cross Matching with GAIA to get ids

In [1]:
import numpy as np
import pandas as pd
import astropy.units as u
import matplotlib.pyplot as plt
from astropy.coordinates import SkyCoord
from astroquery.gaia import Gaia
Gaia.login(user='jadams02', password='LkCa15bcde!')

INFO: Login to gaia TAP server [astroquery.gaia.core]
INFO: OK [astroquery.utils.tap.core]
INFO: Login to gaia data server [astroquery.gaia.core]
INFO: OK [astroquery.utils.tap.core]


## Crossmatch function for GAIA first

In [2]:
def gaia_match_one_with_sep(ra_deg, dec_deg, radius_arcsec=1.0):
    """
    Nearest-neighbor Gaia DR3 match within radius_arcsec.
    Returns (source_id:int, sep_arcsec:float) or (None, None).
    """
    coord = SkyCoord(ra_deg * u.deg, dec_deg * u.deg, frame="icrs")
    job = Gaia.cone_search_async(coord, radius=radius_arcsec * u.arcsec)
    t = job.get_results()

    if len(t) == 0:
        return None, None

    # Nearest match
    t.sort("dist")
    sid = int(t["source_id"][0])

    # Compute separation robustly with astropy (unit-safe)
    gcoord = SkyCoord(t["ra"][0] * u.deg, t["dec"][0] * u.deg, frame="icrs")
    sep_arcsec = float(coord.separation(gcoord).arcsec)

    return sid, sep_arcsec

In [4]:
alma_unique = pd.read_csv("alma_debris_disks_unique_targets.csv")

alma_unique.head()

,s_ra,s_dec,target_name,band_list,t_exptime
0,77.114167,-21.025000,1RXS_J050827.3-210130,7,846.720
1,262.336160,-50.248335,1RXS_J172919.1-501454,7,967.680
2,211.949501,-39.762023,1swasp_j140747.93-394542.6,7,9215.937
3,79.972042,6.290336,2MASS_J05195327+0617258,7,967.680
4,81.079754,-16.020917,2MASS_J05241914-1601153,7,876.960


## Gaia cross match code
Comment out when not using, because it takes 20 minutes to run

In [5]:
ids = []
seps = []

for ra, dec in zip(alma_unique["s_ra"], alma_unique["s_dec"]):
    sid, sep = gaia_match_one_with_sep(ra, dec, radius_arcsec=1.0)
    ids.append(sid)
    seps.append(sep)

alma_unique["gaia_source_id"] = pd.Series(ids, dtype="Int64")   # critical
alma_unique["sep_arcsec"] = pd.Series(seps, dtype="Float64")

alma_unique["has_gaia"] = alma_unique["gaia_source_id"].notna()

alma_unique["gaia_match_quality"] = "good"
alma_unique.loc[alma_unique["sep_arcsec"] > 1.0, "gaia_match_quality"] = "questionable"
alma_unique.loc[alma_unique["sep_arcsec"] > 5.0, "gaia_match_quality"] = "bad"

print(f"Gaia matched: {alma_unique['has_gaia'].sum()} / {len(alma_unique)}")
print("Gaia ID dtype:", alma_unique["gaia_source_id"].dtype)
print(alma_unique.loc[alma_unique["has_gaia"], "gaia_source_id"].head())



INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]


In [6]:
alma_unique["gaia_source_id"].dtype
alma_unique["gaia_source_id"].dropna().head()

1    5949369736173100544
2    6117085769513415168
3    3240927075399243648
4    2983928735054717696
5    5199797745697026560
Name: gaia_source_id, dtype: Int64

In [10]:
alma = alma_unique.copy()

# Ensure Int64 in memory
alma["gaia_source_id"] = alma["gaia_source_id"].astype("Int64")

# Add a string version that will NEVER lose precision on reload
alma["gaia_source_id_str"] = alma["gaia_source_id"].astype("string")

alma.to_csv("alma_debris_unique_with_gaia_safe.csv", index=False)
print("Saved alma_debris_unique_with_gaia_safe.csv")

Saved alma_debris_unique_with_gaia_safe.csv


## Added a quality check 

	•	< 1.0″ = “good” 
	•	1–5″ = “questionable”
	•	> 5″ = “likely wrong / needs manual check”
